--------------------------------------------------------------------------------

# Created by [Nikolaos G. Giakoumoglou](https://github.com/giakou4)

### Electrical and Computer Engineer

Mobile: +30 6906430995 | e-mail: [ngiakoumoglou@gmail.gr](mailto:ngiakoumoglou@gmail.gr) | e-mail: [ngiakoumoglou@hotmail.gr](mailto:ngiakoumoglou@hotmail.gr) |

-------------------------------------------------------------------------------

# YOLOv7

This tutorial is based on the [YOLOv7 repository](https://github.com/WongKinYiu/yolov7) by [Chien-Yao Wang](https://github.com/WongKinYiu)

# 1 Install Dependencies

In [ ]:
%%capture
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

!pip install pycocotools>=2.0
!pip install roboflow

import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
!nvidia-smi

# 2 Download Dataset 
We'll download our dataset from [Roboflow](https://roboflow.com/). Use the "**YOLOv7 PyTorch**" export format.

In [ ]:
%%capture
from roboflow import Roboflow
rf = Roboflow(api_key="7EOGxE0Zhj6CaawbEz1J")
project = rf.workspace("joseph-nelson").project("mask-wearing")
dataset = project.version(4).download("yolov7")

# 3 Train Custom YOLOv7 Detector

To kick off training we running the training command with the following options:

* _img_: define input image size
* _batch_: determine batch size
* _epochs_: define the number of training epochs
* _data_: set the path to our yaml file
* _cfg_: specify our model configuration
* _weights_: specify a custom path to weights
* _name_: result names
* _nosave_: only save the final checkpoint
* _cache_: cache images for faster training
* _hyps_: hyperparameters of training

In [ ]:
img = 1024 #@param {type:"integer"}
batch = 8 #@param {type:"integer"}
epochs = 300 #@param {type:"integer"}
cfg = 'cfg/training/yolov7.yaml' #@param ["cfg/training/yolov7.yaml", "cfg/training/yolov7x.yaml", "cfg/training/yolov7-w6.yaml", "cfg/training/yolov7-e6.yaml", "cfg/training/yolov7-d6.yaml", "cfg/training/yolov7-e6e.yaml"]
weights = 'yolov7_training.pt' #@param ["yolov7_training.pt",  "yolov7x_training.pt", "yolov7-w6_training.pt", "yolov7-e6_training.pt", "yolov7-d6_training.pt", "yolov7-e6e_training.pt"]
hyp = './data/hyp.scratch.p5.yaml' #@param["./data/hyp.scratch.p5.yaml", "./data/hyp.scratch.p6.yaml", "./data/hyp.scratch.tiny.yaml", "./data/hyp.scratch.custom.yaml"]
name = 'experiment-1' #@param {type:"string"}

In [ ]:
%%capture
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/{weights}

In [ ]:
%%time
%cd /content/yolov7
if cfg[13:-5] in ('yolov7', 'yolov7x'):
    print('Train YOLOv7/YOLOv7x')
    !python train.py --img {img} {img} --batch-size {batch} --epochs {epochs} --data {dataset.location}/data.yaml --cfg {cfg} --weights {weights} --hyp {hyp} --device 0 --cache-images --name {name}
elif cfg[13:-5] in ('yolov7-w6', 'yolov7-e6', 'yolov7-d6', 'yolov7-e6e'):
    print('Train Aux: YOLOv7-w6/e6/d6/e6e')
    !python train_aux.py --img-size {img} {img} --batch {batch} --epochs {epochs} --data {dataset.location}/data.yaml --cfg {cfg} --weights {weights} --hyp {hyp} --device 0 --cache-images --name {name}
else:
    print('Invalid cfg')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# 4 Validate Custom YOLOv7 Detector

In [ ]:
!python test.py --data {dataset.location}/data.yaml --img-size {img} --batch {batch} --conf 0.001 --iou 0.65 --device 0 --weights runs/train/{name}/weights/best.pt --name {name} --save-txt --save-conf